# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
M. Güdel  ->  M. Güdel  |  ['M. Güdel']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
T. Ueda  ->  T. Ueda  |  ['T. Ueda']
Arxiv has 57 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.14414


extracting tarball to tmp_2307.14414...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']
L. Boogaard  ->  L. Boogaard  |  ['L. Boogaard']
M. Güdel  ->  M. Güdel  |  ['M. Güdel']


...

✔ → 30889:\section{Triple ALMA bands analysis}\label{sec:analysis}
  ↳ 71907:\section{Discussion}\label{sec:dis}


✔ → 71907:\section{Discussion}\label{sec:dis}
  ↳ 88501:\section{summary}\label{sec:sum}


✔ → 88501:\section{summary}\label{sec:sum}
  ↳ 105795:end


T. Ueda  ->  T. Ueda  |  ['T. Ueda']


list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.14414-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.14414) | **First rest-frame infrared spectrum of a z>7 quasar: JWST/MRS  observations of J1120+0641**  |
|| S. E. I. Bosman, et al. -- incl., <mark>F. Walter</mark>, <mark>L. Boogaard</mark>, <mark>M. Güdel</mark> |
|*Appeared on*| *2023-07-28*|
|*Comments*| *19 pages, 10 figures; submitted*|
|**Abstract**| We present a JWST/MRS spectrum of the quasar J1120+0641 at z=7.0848, the first spectroscopic observation of a reionisation-era quasar in the rest-frame infrared ($0.6<\lambda<3.4\mu$m). In the context of the mysterious fast assembly of the first supermassive black holes at z>7, our observations enable for the first time the detection of hot torus dust, the H$\alpha$ emission line, and the Paschen-series broad emission lines in a quasar at z>7. Hot torus dust is clearly detected as an upturn in the continuum emission at $\lambda_{\text{rest}}\simeq1.3\mu$m, leading to a black-body temperature of $T=1413.5^{+5.7}_{-7.4}$K. Compared to similarly-luminous quasars at 0<z<6, the hot dust in J1120+0641 is somewhat elevated in temperature (top 1%). The temperature is more typical among 6<z<6.5 quasars (top 25%), leading us to postulate a weak evolution in the hot dust temperature at z>6 ($2\sigma$ significance). We measure the black hole mass of J1120+0641 based on the H$\alpha$ Balmer line, $M_{\text{BH}}=1.52\pm0.17\cdot 10^9 M_\odot$, which is in good agreement with the previous rest-UV MgII black hole mass measurement. The black hole mass based on the Paschen-series lines is also consistent, indicating no significant extinction in the rest-frame UV measurement. The broad H$\alpha$, Pa-$\alpha$ and Pa-$\beta$ emission lines are consistent with an origin in a common broad-line region (BLR) with density log$N_H/\text{cm}^{-3}\geq 12$, ionisation parameter $-7<$log$U<-4$, and extinction E(B-V)$\lesssim 0.1$mag. These BLR parameters are consistent with similarly-bright quasars at 0<z<4. Overall, we find that both J1120+0641's hot dust torus and hydrogen BLR properties show no significant peculiarity when compared to luminous quasars down to z=0. The quasar accretion structures must have therefore assembled very quickly, as they appear fully "mature" less than 760 million years after the Big Bang. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.14421-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.14421) | **On the likelihoods of finding very metal-poor (and old) stars in the  Milky Way's disc, bulge, and halo**  |
|| D. Sotillo-Ramos, <mark>M. Bergemann</mark>, J. K. Friske, <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-07-28*|
|*Comments*| *Accepted by MNRAS. 4 figures*|
|**Abstract**| Recent observational studies have uncovered a small number of very metal-poor stars with cold kinematics in the Galactic disc and bulge. However, their origins remain enigmatic. We select a total of 138 Milky Way (MW) analogs from the TNG50 cosmological simulation based on their $z=0$ properties: disky morphology, stellar mass, and local environment. In order to make more predictive statements for the MW, we further limit the spatial volume coverage of stellar populations in galaxies to that targeted by the upcoming 4MOST high-resolution survey of the Galactic disc and bulge. We find that across all galaxies, $\sim$20 per cent of very metal-poor (${\rm [Fe/H]} < -2$) stars belong to the disk, with some analogs reaching 30 per cent. About 50$\pm$10 per cent of the VMP disc stars are, on average, older than 12.5 Gyr and $\sim$70$\pm$10 per cent come from accreted satellites. A large fraction of the VMP stars belong to the halo ($\sim$70) and have a median age of 12 Gyr. Our results with the TNG50 cosmological simulation confirm earlier findings with simulations of fewer individual galaxies, and suggest that the stellar disc of the Milky Way is very likely to host significant amounts of very- and extremely-metal-poor stars that, although mostly of ex situ origin, can also form in situ, reinforcing the idea of the existence of a primordial Galactic disc. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.14526-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.14526) | **Dust enrichment and grain growth in a smooth disk around the DG Tau  protostar revealed by ALMA triple bands frequency observations**  |
|| S. Ohashi, et al. -- incl., <mark>T. Ueda</mark> |
|*Appeared on*| *2023-07-28*|
|*Comments*| *29 pages, 21 figures, 5 tables. Accepted for publication in ApJ*|
|**Abstract**| Characterizing the physical properties of dust grains in a protoplanetary disk is critical to comprehending the planet formation process. Our study presents ALMA high-resolution observations of the young protoplanetary disk around DG Tau at a 1.3 mm dust continuum. The observations, with a spatial resolution of $\approx 0.04''$, or $\approx5$ au, revealed a geometrically thin and smooth disk without substantial substructures, suggesting that the disk retains the initial conditions of the planet formation. To further analyze the distributions of dust surface density, temperature, and grain size, we conducted a multi-band analysis with several dust models, incorporating ALMA archival data of the 0.87 mm and 3.1 mm dust polarization. The results showed that the Toomre $Q$ parameter is $\lesssim2$ at a 20 au radius, assuming a dust-to-gas mass ratio of 0.01. This implies that a higher dust-to-gas mass ratio is necessary to stabilize the disk. The grain sizes depend on the dust models, and for the DSHARP compact dust, they were found to be smaller than $\sim400$ $\mu$m in the inner region ($r\lesssim20$ au), while exceeding larger than 3 mm in the outer part. Radiative transfer calculations show that the dust scale height is lower than at least one-third of the gas scale height. These distributions of dust enrichment, grain sizes, and weak turbulence strength may have significant implications for the formation of planetesimals through mechanisms such as streaming instability. We also discuss the CO snowline effect and collisional fragmentation in dust coagulation for the origin of the dust size distribution. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.14414.md
    + _build/html/tmp_2307.14414/./figures/halpha.png
    + _build/html/tmp_2307.14414/./figures/full_fit_final.png
    + _build/html/tmp_2307.14414/./figures/pgamma.png
exported in  _build/html/2307.14421.md
    + _build/html/tmp_2307.14421/./Figures/histograms_allComponents_massFrac_age_exsitu_MgFe_allGalaxies_massWeighted.png
    + _build/html/tmp_2307.14421/./Figures/lines_multipanels_componentMassFractionAndMassSeparate4Components_vs_age_allGalaxies_geomCut5_5kpc.png
    + _build/html/tmp_2307.14421/./Figures/lines_multipanels_componentMassFractionAndMassSeparate4Components_vs_FeH_allGalaxies_geomCut5_5kpc.png
exported in  _build/html/2307.14526.md
    + _build/html/tmp_2307.14526/./fig8.png
    + _build/html/tmp_2307.14526/./fig18.png
    + _build/html/tmp_2307.14526/./fig9.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# First rest-frame infrared spectrum of a $z>7$ quasar: JWST/MRS observations of J1120+0641

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.14414-b31b1b.svg)](https://arxiv.org/abs/2307.14414)<mark>Appeared on: 2023-07-28</mark> -  _19 pages, 10 figures; submitted_

</div>
<div id="authors">

S. E.~I.~Bosman, et al. -- incl., <mark>F. Walter</mark>, <mark>L. Boogaard</mark>, <mark>M. Güdel</mark>

</div>
<div id="abstract">

**Abstract:** We present a JWST/MRS spectrum of the quasar J1120+0641 at $z=7.0848$ , the first spectroscopic observation of a reionisation-era quasar in the rest-frame infrared wavelengths ( $0.6 < \lambda_\text{rest} < 3.4 \mu$ m).In the context of the mysterious fast assembly of the first supermassive black holes at $z>7$ , our observations enable for the first time the detection of hot torus dust, the H $\alpha$ emission line, and the Paschen-series broad emission lines in a quasar at $z>7$ , which we compare to samples at $z<6$ for signs of evolution.Hot torus dust is clearly detected as an upturn in the continuum emission at $\lambda_{\text{rest}}\simeq1.3\mu$ m, leading to a black-body temperature of $T_\text{dust} = 1413.5_{-7.4}^{+5.7}$ K. Compared to similarly-luminous quasars at $0<z<6$ , the hot dust in J1120+0641 is somewhat elevated in temperature (top $1\%$ ). The temperature is more typical among $6<z<6.5$ quasars (top $25\%$ ), leading us to postulate a weak evolution in the hot dust temperature at $z>6$ ( $2\sigma$ significance).We measure the black hole mass of J1120+0641 based on the H $\alpha$ Balmer line, $M_{\text{BH}} = 1.52 \pm 0.17 \cdot 10^9 M_\odot$ , which is in good agreement with the previous rest-UV Mg ${II}$ black hole mass measurement.The black hole mass based on the Paschen-series lines is also consistent within uncertainties, indicating that no significant extinction is biasing the $M_\text{BH}$ measurement obtained from the rest-frame UV.By comparing the ratios of the H $\alpha$ , Pa- $\alpha$ and Pa- $\beta$ emission lines to predictions from a simple one-phase Cloudy model, we find that the hydrogen broad lines are consistent with originating in a common broad-line region (BLR) with density $\text{log} N_H/\text{cm}^{-3} \geq 12$ , ionisation parameter $-7<\text{log}  U<-4$ , and extinction E(B--V) $\lesssim0.1$ mag. These BLR parameters are fully consistent with similarly-bright quasars at $0<z<4$ .Overall, we find that both J1120+0641's hot dust torus and hydrogen BLR properties show no significant peculiarity when compared to luminous quasars down to $z=0$ . The quasar accretion structures must have therefore assembled very quickly, as they appear fully `mature' less than $760$ million years after the Big Bang.

</div>

<div id="div_fig1">

<img src="tmp_2307.14414/./figures/halpha.png" alt="Fig6" width="100%"/>

**Figure 6. -** Spectrum of the H$\alpha$ broad line (flux in black and uncertainty in grey) with the optimal line model fit (orange), split into the continuum constituting of a power-law and a black-body (green) and the broad emission lines (red). (*fig:ha*)

</div>
<div id="div_fig2">

<img src="tmp_2307.14414/./figures/full_fit_final.png" alt="Fig8" width="100%"/>

**Figure 8. -** Full fit to the MIRI-MRS spectrum of J1120+0641 (top, orange) and its residuals (bottom) in units of standard deviations. The coloured lines show the emission coming from the accretion disk (power law, blue), the hot torus dust (black-body radiation, green) and the broad quasar emission lines (red). The residuals show no signs of deviation from the continuum model, nor additional broad emission lines. The gray line is an in Fig. \ref{fig:spec} and the red dashed line indicates the location of zero residuals. See Fig. \ref{fig:spec} for line IDs. (*fig:fit*)

</div>
<div id="div_fig3">

<img src="tmp_2307.14414/./figures/pgamma.png" alt="Fig3" width="100%"/>

**Figure 3. -** Spectrum of the Pa-$\delta$, He {I} and Pa-$\gamma$ broad lines (in order of increasing wavelength) and the optimal model fit. The legend is the same as in Figure \ref{fig:ha}. (*fig:pg*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.14414"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\DSR}[1]{\textcolor{red}{#1}}$
$\newcommand{\DSRc}[1]{\textit{\textcolor{red}{#1}}}$
$\newcommand{\DTA}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\ap}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\apn}[1]{\textcolor{red}{#1}}$
$\newcommand{\mb}[1]{\textcolor{green}{#1}}$
$\newcommand{\MS}{\rm{M}_{\odot}}$
$\newcommand{\MH}{\rm{M}_{\rm 200c}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\}{blankpage}$</div>



<div id="title">

# On the likelihoods of finding very metal-poor (and old) stars in the Milky Way's disc, bulge, and halo

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.14421-b31b1b.svg)](https://arxiv.org/abs/2307.14421)<mark>Appeared on: 2023-07-28</mark> -  _Accepted by MNRAS. 4 figures_

</div>
<div id="authors">

D. Sotillo-Ramos, <mark>M. Bergemann</mark>, J. K. Friske, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** Recent observational studies have uncovered a small number of very metal-poor stars with cold kinematics in the Galactic disc and bulge. However, their origins remain enigmatic. We select a total of 138 Milky Way (MW) analogs from the  TNG50 cosmological simulation based on their $z=0$ properties: disky morphology, stellar mass, and local environment. In order to make more predictive statements for the MW, we further limit the spatial volume coverage of stellar populations in galaxies to that targeted by the upcoming 4MOST high-resolution survey of the Galactic disc and bulge. We find that across all galaxies, $\sim$ 20 per cent of very metal-poor ( ${\rm[Fe/H]} < -2$ ) stars belong to the disk, with some analogs reaching 30 per cent. About 50 $\pm$ 10 per cent of the VMP disc stars are, on average, older than 12.5 Gyr and $\sim$ 70 $\pm$ 10 per cent come from accreted satellites. A large fraction of the VMP stars belong to the halo ( $\sim$ 70) and have a median age of 12 Gyr. Our results with the TNG50 cosmological simulation confirm earlier findings with simulations of fewer individual galaxies, and suggest that the stellar disc of the Milky Way is very likely to host significant amounts of very- and extremely-metal-poor stars that, although mostly of ex situ origin, can also form in situ, reinforcing the idea of the existence of a primordial Galactic disc.

</div>

<div id="div_fig1">

<img src="tmp_2307.14421/./Figures/histograms_allComponents_massFrac_age_exsitu_MgFe_allGalaxies_massWeighted.png" alt="Fig4" width="100%"/>

**Figure 4. -**  Mass fraction per component, stellar age, [Mg/Fe] and ex situ fraction distributions, for stellar samples with different values of [Fe/H]. We show the distributions of the median values across all TNG50 138 MW-like galaxies weighted by stellar mass and do not apply a heliocentric cut. (*fig:histograms*)

</div>
<div id="div_fig2">

<img src="tmp_2307.14421/./Figures/lines_multipanels_componentMassFractionAndMassSeparate4Components_vs_age_allGalaxies_geomCut5_5kpc.png" alt="Fig3" width="100%"/>

**Figure 3. -** Same as Fig. 2, but for the distribution of stellar ages in the morphological components. _Top_: Stellar mass fraction per component to total stellar mass. _Middle_: Stellar mass per component. _Bottom_: Mass of MP stars per component. The vertical dashed lines represent the median age of the stars in the component (within the volumetric cut), across all galaxies. (*fig:3*)

</div>
<div id="div_fig3">

<img src="tmp_2307.14421/./Figures/lines_multipanels_componentMassFractionAndMassSeparate4Components_vs_FeH_allGalaxies_geomCut5_5kpc.png" alt="Fig2" width="100%"/>

**Figure 2. -** Metallicity distributions of stars in TNG50 MW-like galaxies, grouped by their respective morphological component: cold disc, warm disc, bulge and stellar halo, from left to right. The top panels quantify the stellar mass fractions in each component to the total stellar mass; the bottom ones are the metallicity distributions in each morphological component, in stellar mass. The solid lines represent the medians and the shaded areas and error-bars represent inter-per centile ranges across the galaxy sample: 16 to 84 and 2 to 98, respectively. The vertical red shaded bands highlight metallicities [Fe/H]$\leq -2$, i.e., VMP stars. We remind the reader that the distributions are shown for the characteristic observable volume fraction of the Galaxy, as it will be `seen' by the 4MIDABLE-HR survey on 4MOST (see Section \ref{sec:morpho}). Specifically, we apply a volume cut of $5.5$ kpc in heliocentric distance, where the fiducial 'Sun' is placed at $8$ kpc in the simulated galaxy.  (*fig:2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.14421"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\usepackage{amsmath}$
$\usepackage{longtable}$
$\newcommand{\pd}{\partial}$
$\newcommand{\AU}{{\rm AU}}$
$\newcommand{\kB}{k_{\rm B}}$
$\shorttitle{Discovery of dust enrichment and grain growth in a smooth DG Tau disk }$
$\shortauthors{Ohashi et al.}$
$\graphicspath{{./}{figures/}}$
$\begin{document}$
$\title{Dust enrichment and grain growth in a smooth disk around the DG Tau protostar revealed by ALMA triple bands frequency observations}$
$\author[0000-0002-9661-7958]{Satoshi Ohashi}$
$\altaffiliation{NAOJ Fellow}$
$\affil{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo 181-8588, Japan}$
$\affil{RIKEN Cluster for Pioneering Research, 2-1 Hirosawa, Wako-shi, Saitama 351-0198, Japan}$
$\email{satoshi.ohashi@nao.ac.jp}$
$\author[0000-0002-3001-0897]{Munetake Momose}$
$\affiliation{College of Science, Ibaraki University, 2-1-1 Bunkyo, Mito, Ibaraki 310-8512, Japan}$
$\author[0000-0003-4562-4119]{Akimasa Kataoka}$
$\affiliation{National Astronomical Observatory of Japan, 2-21-1 Osawa, Mitaka, Tokyo 181-8588, Japan}$
$\author[0000-0002-9221-2910]{Aya E Higuchi}$
$\affil{Division of Science, School of Science and Engineering, Tokyo Denki University, Ishizaka, Hatoyama-machi, Hiki-gun, Saitama 350-0394, Japan}$
$\author[0000-0002-6034-2892]{Takashi Tsukagoshi}$
$\affiliation{Faculty of Engineering, Ashikaga University, Ohmae-cho 268-1, Ashikaga, Tochigi 326-8558, Japan}$
$\author[0000-0003-4902-222X]{Takahiro Ueda}$
$\affiliation{Max-Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\author[0000-0003-1514-3074]{Claudio Codella}$
$\affil{INAF, Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Firenze, Italy}$
$\affil{Univ. Grenoble Alpes, CNRS, IPAG, 38000 Grenoble, France}$
$\author{Linda Podio}$
$\affiliation{INAF, Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Firenze, Italy}$
$\author[0000-0002-7538-581X]{Tomoyuki Hanawa}$
$\affil{Center for Frontier Science, Chiba University, 1-33 Yayoi-cho, Inage-ku, Chiba, Chiba 263-8522, Japan}$
$\author[0000-0002-3297-4497]{Nami Sakai}$
$\affil{RIKEN Cluster for Pioneering Research, 2-1 Hirosawa, Wako-shi, Saitama 351-0198, Japan}$
$\author[0000-0001-8808-2132]{Hiroshi Kobayashi}$
$\affil{Department of Physics, Graduate School of Science, Nagoya University, Furo-cho, Chikusa-ku, Nagoya 464-8602, Japan}$
$\author[0000-0002-1886-0880]{Satoshi Okuzumi}$
$\affil{Department of Earth and Planetary Sciences, Tokyo Institute of Technology, Meguro-ku, Tokyo 152-8551, Japan}$
$\author[0000-0001-9659-658X]{Hidekazu Tanaka}$
$\affil{Astronomical Institute, Tohoku University, 6-3 Aramaki, Aoba-ku, Sendai 980-8578, Japan}$
$\begin{abstract}$
$Characterizing the physical properties of dust grains in a protoplanetary disk is critical to comprehending the planet formation process. Our study presents ALMA high-resolution observations of the young protoplanetary disk around DG Tau at a 1.3 mm dust continuum. The observations, with a spatial resolution of \approx0\farcs04, or \approx5 au, revealed a geometrically thin and smooth disk without substantial substructures, suggesting that the disk retains the initial conditions of the planet formation. To further analyze the distributions of dust surface density, temperature, and grain size, we conducted a multi-band analysis with several dust models, incorporating ALMA archival data of the 0.87 mm and 3.1 mm dust polarization. The results showed that the Toomre {\it Q} parameter is \lesssim2 at a 20 au radius, assuming a dust-to-gas mass ratio of 0.01. This implies that a higher dust-to-gas mass ratio is necessary to stabilize the disk. The grain sizes depend on the dust models, and for the DSHARP compact dust, they were found to be smaller than \sim400 \mum in the inner region (r\lesssim20 au), while exceeding larger than 3 mm in the outer part. Radiative transfer calculations show that the dust scale height is lower than at least one-third of the gas scale height. These distributions of dust enrichment, grain sizes, and weak turbulence strength may have significant implications for the formation of planetesimals through mechanisms such as streaming instability.  We also discuss the CO snowline effect and collisional fragmentation in dust coagulation for the origin of the dust size distribution.$
$\end{abstract}$
$\n\end{document}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\pd}{\partial}$
$\newcommand{\AU}{{\rm AU}}$
$\newcommand{\kB}{k_{\rm B}}$</div>



<div id="title">

# Dust enrichment and grain growth in a smooth disk around the DG Tau protostar revealed by ALMA triple bands frequency observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.14526-b31b1b.svg)](https://arxiv.org/abs/2307.14526)<mark>Appeared on: 2023-07-28</mark> -  _29 pages, 21 figures, 5 tables. Accepted for publication in ApJ_

</div>
<div id="authors">

S. Ohashi, et al. -- incl., <mark>T. Ueda</mark>

</div>
<div id="abstract">

**Abstract:** Characterizing the physical properties of dust grains in a protoplanetary disk is critical to comprehending the planet formation process. Our study presents ALMA high-resolution observations of the young protoplanetary disk around DG Tau at a 1.3 mm dust continuum. The observations, with a spatial resolution of $\approx0\farcs04$ , or $\approx5$ au, revealed a geometrically thin and smooth disk without substantial substructures, suggesting that the disk retains the initial conditions of the planet formation. To further analyze the distributions of dust surface density, temperature, and grain size, we conducted a multi-band analysis with several dust models, incorporating ALMA archival data of the 0.87 mm and 3.1 mm dust polarization. The results showed that the Toomre $_ Q_$ parameter is $\lesssim2$ at a 20 au radius, assuming a dust-to-gas mass ratio of 0.01. This implies that a higher dust-to-gas mass ratio is necessary to stabilize the disk. The grain sizes depend on the dust models, and for the DSHARP compact dust, they were found to be smaller than $\sim400$ $\mu$ m in the inner region ( $r\lesssim20$ au), while exceeding larger than 3 mm in the outer part. Radiative transfer calculations show that the dust scale height is lower than at least one-third of the gas scale height. These distributions of dust enrichment, grain sizes, and weak turbulence strength may have significant implications for the formation of planetesimals through mechanisms such as streaming instability.  We also discuss the CO snowline effect and collisional fragmentation in dust coagulation for the origin of the dust size distribution.

</div>

<div id="div_fig1">

<img src="tmp_2307.14526/./fig8.png" alt="Fig11" width="100%"/>

**Figure 11. -** Posterior probability distributions of the fitting parameters for the multiband observations. The marginalized posterior probability is normalized by the maximum at each radius.
The black  line in the middle panel shows the dust surface density profile where the disk is gravitationally unstable (i.e., $Q = 1$), assuming a gas-to-dust ratio of 100 and the higher temperature in the highest probabilities.
The red dashed lines and pink dotted lines show the models discussed in Section \ref{sec:model1} and \ref{sec:model2}, respectively.
 (*probability*)

</div>
<div id="div_fig2">

<img src="tmp_2307.14526/./fig18.png" alt="Fig5" width="100%"/>

**Figure 5. -** Posterior probability distribution of the model parameters fitted to the multi-band observations at a disk radius of 20 au in the DSHARP compact dust.
The parameter space is $\Sigma_{\rm d}$, $T$, and $a_{\rm max}$ but the maximum probabilities in the parameter range of $a_{\rm max} = 10$$\mu$m $-$ 10 cm are shown.
 (*qat20au*)

</div>
<div id="div_fig3">

<img src="tmp_2307.14526/./fig9.png" alt="Fig12" width="100%"/>

**Figure 12. -** Comparison of radial profiles between the observed intensities and model intensities.  The model is described in Section \ref{sec:model1}. The black solid lines indicate the observed intensities with uncertainties of the rms noise levels and the flux calibration errors, shown as the shaded region.
The red solid, cyan dashed, and purple dashed lines show the model intensities with dust sizes of 1 cm, 3 mm, and 1 mm in the outer region of the disk ($r\geq40$ au), respectively.
 (*model1_profile*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.14526"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

240  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
